In [1]:
import psycopg2
import pandas as pd
import logging
from dotenv import load_dotenv
import os

In [2]:
# Set up logging
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger()
# create a log file
fh = logging.FileHandler("log.log")
fh.setLevel(logging.INFO)
logger.addHandler(fh)

In [3]:
# Load .env file
load_dotenv()

# Get database connection parameters from environment variables
dbname = os.getenv("DB_NAME")
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")

In [4]:
# Check current working directory
current_working_directory = os.getcwd()
logger.info(f"Current Working Directory: {current_working_directory}")
print(f"Current Working Directory: {current_working_directory}")

2024-06-10 18:51:41,762 - INFO - Current Working Directory: /Users/nicksolly/Dev/outreachOptimisation


Current Working Directory: /Users/nicksolly/Dev/outreachOptimisation


In [7]:
file_path = './inputData/PreviousWeeksEngagementLists/20240510 Engagement List.xlsx'
# logger.info(f"File Path: {file_path}")

In [ ]:
# Database connection
conn = psycopg2.connect(
    dbname=dbname,
    user=user,
    password=password,
    host=host,
    port=port
)

In [8]:
# Load the Excel data into a DataFrame
df_raw = pd.read_excel(file_path)
# add to logger with information of data loaded (e.g. shape, columns, etc.)
logger.info(f"Data loaded with shape: {df_raw.shape}")
# logger.info(f'Columns: {df_raw.columns}')

2024-06-10 18:52:29,389 - INFO - Data loaded with shape: (19327, 312)


In [11]:
# get unique values of the 'Engagement Partner Service Line' column
unique_service_lines = df_raw['Engagement Partner Service Line'].unique()
print(f"Unique Service Lines: {unique_service_lines}")

Unique Service Lines: ['CBS & Elim' 'Assurance' 'Consulting' nan 'Tax' 'Undefined' 'SaT']


In [ ]:
# Filter the data
df_filtered = df_raw[
    (df_raw["Engagement Partner Service Line"] == "Consulting")
    & (df_raw["Engagement Status"] == "Released")
]
# add to logger with information of df_raw filtered (e.g. shape, columns, etc.)
logger.info(f"data filtered with shape: {df_filtered.shape}")

In [ ]:
df_filtered.head()
# reduce columns to only the ones needed
keep_cols = [
    "Engagement ID",
    "Creation Date",
    "Release Date",
    "Last Time Charged Date",
    "Last Expenses Charged Date",
    "Last Active ETC-P Date",
    "Engagement",
    "Client",
    "Engagement Partner",
    "Engagement Partner GUI",
    "Engagement Manager",
    "Engagement Manager GUI",
]

df_filtered= df_filtered[keep_cols]
# add to logger with information of data reduced (e.g. shape, columns, etc.)
logger.info(f"Data reduced with shape: {df_filtered.shape}")
df_filtered.head()

In [ ]:
df_filtered['Last Time Charged Date'].dtypes

# Convert to datetime
df_filtered["Last Time Charged Date"] = pd.to_datetime(df_filtered["Last Time Charged Date"])
# repeat for other date columns Creation Date, Release Date, Last Expenses Charged Date, Last Active ETC-P Date
df_filtered["Creation Date"] = pd.to_datetime(df_filtered["Creation Date"])
df_filtered["Release Date"] = pd.to_datetime(df_filtered["Release Date"])
df_filtered["Last Expenses Charged Date"] = pd.to_datetime(df_filtered["Last Expenses Charged Date"])
df_filtered["Last Active ETC-P Date"] = pd.to_datetime(df_filtered["Last Active ETC-P Date"])


In [ ]:
# add temp calculated columns:
# fill blank values in Last ACTIVE ETC-P Date with Release Date
df_filtered.loc[:, "Last ETC Date"] = df_filtered["Last Active ETC-P Date"].fillna(
    df_filtered["Release Date"]
)

# Ensure 'Data Date' and 'Last ETC Date' columns are in datetime format
df_filtered.loc[:, "Data Date"] = df_filtered["Last Time Charged Date"].max()

# ETC status calculation


# convert last ETC date to datetime
df_filtered["Last ETC Date"] = pd.to_datetime(df_filtered["Last ETC Date"])

# print data types
logger.info(f"Data Types: {df_filtered.dtypes}")

In [ ]:

# Calculate the age of ETC in days using date offset
df_filtered.loc[:, "ETC Age"] = (
    df_filtered["Data Date"] - df_filtered["Last ETC Date"]
).dt.days

df_filtered.loc[:, "Data Date"] = df_filtered["Last Time Charged Date"].max()

# reset index
df_filtered.reset_index(drop=True, inplace=True)

df_filtered.head()

In [ ]:
# are any value null in LAST ETC DATE?
logger.info(f"Any null values in 'Last ETC Date' column: {df_filtered['Last ETC Date'].isnull().values.any()}")

In [ ]:
# create a new table in the database
create_table_query = """
CREATE TABLE IF NOT EXISTS engagement_list (
    engagement_id VARCHAR(255),
    creation_date DATE,
    release_date DATE,
    last_time_charged_date DATE,
    last_expenses_charged_date DATE,
    last_active_etc_p_date DATE,
    engagement VARCHAR(255),
    client VARCHAR(255),
    engagement_partner VARCHAR(255),
    engagement_partner_gui VARCHAR(255),
    engagement_manager VARCHAR(255),
    engagement_manager_gui VARCHAR(255),
    last_etc_date DATE,
    data_date DATE,
    etc_age INT
);
"""
# create a cursor object
cur = conn.cursor()
# execute the query to create the table
cur.execute(create_table_query)
# commit the transaction
conn.commit()

In [ ]:
# Function to handle NaT values
def handle_nat(value):
    if pd.isna(value):
        return None
    return value

# Insert filtered data into the table with logging
for i, row in df_filtered.iterrows():
    try:
        # Check if the row already exists
        cur.execute(
            """
            SELECT 1 FROM engagement_list
            WHERE engagement_id = %s AND data_date = %s
            """,
            (row["Engagement ID"], row["Data Date"])
        )
        if cur.fetchone():
            logger.info(
                f"Skipping row {i + 1}/{len(df_filtered)}: {row['Engagement ID']} (already exists)"
            )
            continue

        # Insert the row if it does not exist
        cur.execute(
            """
            INSERT INTO engagement_list (
                engagement_id, creation_date, release_date, last_time_charged_date, last_expenses_charged_date,
                last_active_etc_p_date, engagement, client, engagement_partner, engagement_partner_gui,
                engagement_manager, engagement_manager_gui, last_etc_date, data_date, etc_age
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """,
            (
                row["Engagement ID"],
                handle_nat(row["Creation Date"]),
                handle_nat(row["Release Date"]),
                handle_nat(row["Last Time Charged Date"]),
                handle_nat(row["Last Expenses Charged Date"]),
                handle_nat(row["Last Active ETC-P Date"]),
                row["Engagement"],
                row["Client"],
                row["Engagement Partner"],
                row["Engagement Partner GUI"],
                row["Engagement Manager"],
                row["Engagement Manager GUI"],
                handle_nat(row["Last ETC Date"]),
                row["Data Date"],
                row["ETC Age"],
            ),
        )
        logger.info(
            f"Inserted row {i + 1}/{len(df_filtered)}: {row['Engagement ID']}"
        )
    except psycopg2.Error as e:
        logger.error(f"Error inserting row {i + 1}: {e}")
        conn.rollback()  # Rollback the transaction to reset the state

# Commit changes and close the connection
conn.commit()

logger.info("Data insertion completed.")

In [ ]:
# query to number of rows to check the data in the table created in the database
cur = conn.cursor()
cur.execute("SELECT COUNT(*) FROM engagement_list")
rows = cur.fetchone()
logger.info(f"Number of rows in the table: {rows[0]}")